In [5]:
!pip install tensorflow  # This will install TensorFlow along with a compatible numpy version
!pip install transformers==4.33.0 tokenizers datasets accelerate
# Install tts with a version compatible with the numpy version required by TensorFlow, if available
# !pip install tts==compatible_version_here



  Using cached tensorflow-2.15.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached flatbuffers-23.5.26-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.5.4-py3-none-any.whl.metadata (1.3 kB)
  Using cached h5py-3.10.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.5 kB)
  Using cached libclang-16.0.6-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached ml_dtypes-0.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached termcolor-2.4.0-py3-none-any.whl.metadata (6.1 kB)
  Using cached tensorflow_io_gcs_filesystem-0.36.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (14 kB)
  Using cached tensorboard-2.15.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached tensorflow_estimator-2.15.0-py2.py3-none-any.whl.metadata (1.3 kB)


In [6]:
import tensorflow as tf
import glob
import os
import shutil
import tqdm
import random
import matplotlib.pyplot as plt
import torch
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from transformers import PreTrainedTokenizerFast
from transformers import DataCollatorForLanguageModeling
from transformers import GPT2Config, GPT2LMHeadModel
from transformers import TrainingArguments, Trainer

tf.config.list_physical_devices("GPU")

2024-02-29 13:38:07.596521: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 13:38:07.596543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 13:38:07.597297: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-29 13:38:07.601584: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-29 13:38:08.486756: W tensorflow/compiler/tf2

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

import os
import re
import glob

# Define the source and target directories
source_dir = '../data/input/clean_data'
target_dir = '../data/input/filtered_data'
dataset_file = os.path.join(target_dir, "filtered_dataset.txt")

# Ensure the target directory exists
os.makedirs(target_dir, exist_ok=True)

# Regex pattern to keep English letters, Pali diacritics, and standard punctuation
pattern = re.compile(r"[^a-zA-ZāīūṅñṭḍṇḷĀĪŪṄÑṬḌṆḶ\s.,!?()-]")

# Function to clean text by removing unwanted characters
def clean_text(text):
    return pattern.sub('', text)

# Find all .txt files in the source directory
paths = glob.glob(os.path.join(source_dir, "*.txt"))[:1000]  # Limit to first 1000 files

# Process and merge content of each file
with open(dataset_file, "w", encoding='utf-8') as output_file:
    for path in paths:
        with open(path, "r", encoding='utf-8') as file:
            for line in file:
                cleaned_line = clean_text(line.strip())
                if cleaned_line:
                    print(cleaned_line, file=output_file)

print("Corpus loaded and filtered.")


import os
import re
import glob
import shutil

# Define the source directories for copying .txt files
source_dirs = [
    "../data/input/text/books_converted",
    "../data/input/text/web_txt",
    "../data/input/audio_transcribed/transcribed_youtube",
    "../data/input/audio_transcribed/transscribed_audio_dharma"
]

# Define the target directory for copied .txt files
copy_target_dir = '../data/input/clean_data'

# Ensure the target directory exists
os.makedirs(copy_target_dir, exist_ok=True)

# Copy all .txt files from the source directories to the target directory
for src_dir in source_dirs:
    for txt_file in glob.glob(os.path.join(src_dir, "*.txt")):
        shutil.copy(txt_file, copy_target_dir)

# Now, set the source_dir to where we've copied the .txt files
source_dir = copy_target_dir

# Define the target directory for preprocessing output
target_dir = '../data/input/filtered_data'
dataset_file = os.path.join(target_dir, "filtered_dataset.txt")

# Ensure the preprocessing target directory exists
os.makedirs(target_dir, exist_ok=True)

# Regex pattern to keep English letters, Pali diacritics, and standard punctuation
pattern = re.compile(r"[^a-zA-ZāīūṅñṭḍṇḷĀĪŪṄÑṬḌṆḶ\s.,!?()-]")

# Function to clean text by removing unwanted characters
def clean_text(text):
    return pattern.sub('', text)

# Find all .txt files in the source directory for preprocessing
paths = glob.glob(os.path.join(source_dir, "*.txt"))[:1000]  # Limit to first 1000 files

# Process and merge content of each file
with open(dataset_file, "w", encoding='utf-8') as output_file:
    for path in paths:
        with open(path, "r", encoding='utf-8') as file:
            for line in file:
                cleaned_line = clean_text(line.strip())
                if cleaned_line:
                    print(cleaned_line, file=output_file)

print("Corpus loaded and filtered.")


# Clean Dataset: Convert to English Sentences

import spacy
import re

# Load the English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

def is_likely_english(sentence):
    # Check if the sentence has at least one verb and one subject, common in English sentences
    has_verb = any(token.pos_ == 'VERB' for token in sentence)
    has_subject = any(token.dep_ == 'nsubj' or token.dep_ == 'nsubjpass' for token in sentence)
    return has_verb and has_subject

def is_not_reference(line):
    # Filter out lines that resemble references or citations, e.g., "SN V," "T II b"
    return not re.search(r'\b(SN|T|MN|AN|D|SĀ|Ps|Vin|Vism)\s+[IVXLC]+', line)

def filter_sentences(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', encoding='utf-8') as f_out:
        for line in f_in:
            if is_not_reference(line):
                doc = nlp(line.strip())
                for sent in doc.sents:
                    if is_likely_english(sent) and len(sent.text.split()) > 5:  # Minimum word count check
                        f_out.write(sent.text + '\n')

# Specify the path to your input file and the desired output file
input_file = '../data/input/filtered_data/filtered_dataset.txt'
output_file = '../data/input/filtered_data/cleaned_dataset.txt'

# Filter the dataset
filter_sentences(input_file, output_file)

print("Dataset filtering complete. Check the cleaned dataset at:", output_file)



# Combined Cleaning and Filtering of Dataset

import os
import re
import glob
import shutil
import spacy

# Define the source directories for copying .txt files
source_dirs = [
    "../data/input/text/books_converted",
    "../data/input/text/web_txt",
    "../data/input/audio_transcribed/transcribed_youtube",
    "../data/input/audio_transcribed/transscribed_audio_dharma"
]

# Define the first target directory for copied .txt files
copy_target_dir = '../data/input/clean_data'

# Ensure the first target directory exists
os.makedirs(copy_target_dir, exist_ok=True)

# Copy all .txt files from the source directories to the first target directory
for src_dir in source_dirs:
    for txt_file in glob.glob(os.path.join(src_dir, "*.txt")):
        shutil.copy(txt_file, copy_target_dir)

# Define the target directory for preprocessing output and the output file
target_dir = '../data/input/filtered_data'
dataset_file = os.path.join(target_dir, "filtered_dataset.txt")

# Ensure the preprocessing target directory exists
os.makedirs(target_dir, exist_ok=True)

# Regex pattern to keep English letters, Pali diacritics, and standard punctuation
pattern = re.compile(r"[^a-zA-ZāīūṅñṭḍṇḷĀĪŪṄÑṬḌṆḶ\s.,!?()-]")

# Function to clean text by removing unwanted characters
def clean_text(text):
    return pattern.sub('', text)

# Find and preprocess .txt files in the first target directory
paths = glob.glob(os.path.join(copy_target_dir, "*.txt"))[:1000]  # Limit to first 1000 files

# Process and merge content of each file
with open(dataset_file, "w", encoding='utf-8') as output_file:
    for path in paths:
        with open(path, "r", encoding='utf-8') as file:
            for line in file:
                cleaned_line = clean_text(line.strip())
                if cleaned_line:
                    print(cleaned_line, file=output_file)

# Load the English tokenizer, tagger, parser, NER, and word vectors from spaCy
nlp = spacy.load("en_core_web_sm")

def is_likely_english(sentence):
    # Check if the sentence has at least one verb and one subject
    has_verb = any(token.pos_ == 'VERB' for token in sentence)
    has_subject = any(token.dep_ == 'nsubj' or token.dep_ == 'nsubjpass' for token in sentence)
    return has_verb and has_subject

def is_not_reference(line):
    # Filter out lines that resemble references or citations
    return not re.search(r'\b(SN|T|MN|AN|D|SĀ|Ps|Vin|Vism)\s+[IVXLC]+', line)

def filter_sentences(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as f_in, open(output_file, 'w', encoding='utf-8') as f_out:
        for line in f_in:
            if is_not_reference(line):
                doc = nlp(line.strip())
                for sent in doc.sents:
                    if is_likely_english(sent) and len(sent.text.split()) > 5:  # Minimum word count check
                        f_out.write(sent.text + '\n')

# Specify the path for the final output file
final_output_file = '../data/input/filtered_data/cleaned_dataset.txt'

# Filter the preprocessed dataset
filter_sentences(dataset_file, final_output_file)

print("Corpus loaded, filtered, and cleaned. Check the cleaned dataset at:", final_output_file)


# Produce Descriptive Statistics

In [7]:
import re

def format_number(number):
    """Format the number with underscores between hundreds."""
    return f"{number:,}".replace(",", "_")

def count_words(file_path):
    """Count words in a file and provide descriptive statistics."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
            # Tokenize the text by non-alphanumeric characters
            words = re.findall(r'\b\w+\b', text.lower())
            word_count = len(words)
            unique_words = len(set(words))
            average_word_length = sum(len(word) for word in words) / word_count

            print(f"Total Words: {format_number(word_count)}")
            print(f"Unique Words: {format_number(unique_words)}")
            print(f"Average Word Length: {average_word_length:.2f} characters")
    except FileNotFoundError:
        print(f"The file at {file_path} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    file_path = "../data/input/filtered_data_open_source/cleaned_filtered_corrected_dataset.txt"
    count_words(file_path)



Total Words: 2_108_767
Unique Words: 26_120
Average Word Length: 4.18 characters


In [8]:

target_dir = '../data/input/filtered_data_open_source/'
dataset_file = os.path.join(target_dir, "cleaned_filtered_corrected_dataset.txt")

raw_datasets = load_dataset("text", data_files=dataset_file)
raw_datasets
# 333_056 lines
# previously around 43_000 lines

Generating train split: 112985 examples [00:00, 446730.44 examples/s]


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 112985
    })
})

In [9]:
raw_datasets["train"][666]
# with HuggingFace datasets, every sample is a dictionary
# a sample from the dataset here is a paragraph
# the key is always "text"

{'text': "Oh yes, one of the things which is said also is that it's a pure bright mind."}

In [10]:
# Create empty tokenizer and its trainer
tokenizer = Tokenizer(BPE(unk_token="[UNK]")) # subword tokenization and ways to merge them
trainer = BpeTrainer(vocab_size=5_000, special_tokens=["[UNK]", "[PAD]"])
# separates the tokens with a space
tokenizer.pre_tokenizer = Whitespace()

# Batch samples to speed up process
def batch_iterator(batch_size=1000):
    # the batch size is the number of samples that will be processed at once
    # the iterator will yield a batch of samples
    # yield is a keyword in Python that is used like return, except the function will return a generator
    # a generator is an iterator that generates one item at a time
    for i in range(0, len(raw_datasets["train"]), batch_size):
        yield raw_datasets["train"][i : i + batch_size]["text"]

# Train the tokenizer
tokenizer.train_from_iterator(batch_iterator(), trainer=trainer, length=len(raw_datasets["train"]))

# Saves the tokenizer
# when downloading model, we download model, and the tokenizer
tokenizer.save("tokenizer.json")

# Load it fast
# speeds up the process
tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer.json") 
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

0

# Tokenize the Dataset

In [11]:
# start with sequence length of 256
# pads the sequences to the same length
sequence_length = 256

# takes a dictionary as input
def tokenize_function(example):
    # tokenize the text
    tokenized_example = tokenizer(
        example["text"],
        truncation=True,
        padding=True,
        max_length=sequence_length,
    )
    return {"input_ids": tokenized_example["input_ids"]}

# tokenize entire dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True, remove_columns=raw_datasets["train"].column_names)

Map: 100%|██████████| 112985/112985 [00:04<00:00, 25023.85 examples/s]


In [12]:
# data collator is used to batch the samples together
# data pump for training
# collate means to collect and combine
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Make the Model

In [13]:
model_config = GPT2Config(
    vocab_size=tokenizer.vocab_size, # the size of the vocabulary
    pad_token_id=tokenizer.pad_token_id, # the token id for padding
    n_ctx=sequence_length, # context length
    n_positions=sequence_length, # positions in context, the order of the tokens in sequence
    n_embd=512, # embedding dimension
    n_head=8, # number of heads in the multi-head attention models
    n_layer=6, # number of layers
)

model = GPT2LMHeadModel(model_config)
model

# wte: word token embeddings; which means the embeddings of the tokens
# wpe: word position embeddings; which means the embeddings of the positions of the tokens
# drop: dropout
# sequence length: 256
# dropout layer: 0.1
# dropout is a regularization technique; it prevents overfitting
# normalisation is done first, which differs to transformers
# normalisation means to scale the input to have a mean of 0 and a standard deviation of 1
# Conv1D means 1D convolution; convolutions are used to extract features from the input
# LayerNorm means layer normalisation; normalisation is used to improve the training of the model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(5000, 512)
    (wpe): Embedding(256, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=512, out_features=5000, bias=False)
)

In [14]:
# clear VRAM GPU cache
torch.cuda.empty_cache()

output_path = "output"

# Create the Trainer

training_args = TrainingArguments(
    output_dir=output_path, # output directory
    overwrite_output_dir=True, # overwrite the content of the output directory
    num_train_epochs=10, # number of training epochs
    #per_device_train_batch_size=16, # batch size for training per device (e.g. multiple GPUs), which took 8 minutes
    #per_device_train_batch_size=32 # double, because i was only using around 3GB of VRAM, which took 7.5 minutes
    per_device_train_batch_size=52 # increase, because i was only using around 6GB of VRAM with 32, which also took 7.5 minutes
    # now uses 7704MiB of VRAM
)

trainer = Trainer(
    model=model, # the model
    args=training_args, # training arguments
    data_collator=data_collator, # data collator
    train_dataset=tokenized_datasets["train"] # training dataset
)

# Train
trainer.train()

# Save
tokenizer.save_pretrained(output_path)
model.save_pretrained(output_path)

# TODO:
# You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  2%|▏         | 500/21730 [04:14<3:00:24,  1.96it/s]

{'loss': 5.5573, 'learning_rate': 4.884951679705477e-05, 'epoch': 0.23}


  5%|▍         | 1000/21730 [08:31<2:58:32,  1.94it/s]

{'loss': 4.9472, 'learning_rate': 4.769903359410953e-05, 'epoch': 0.46}


  7%|▋         | 1500/21730 [12:56<2:54:48,  1.93it/s]

...

{'loss': 3.5081, 'learning_rate': 1.6797054763000461e-06, 'epoch': 9.66}


 99%|█████████▉| 21500/21730 [3:10:02<02:00,  1.91it/s]

{'loss': 3.4954, 'learning_rate': 5.29222273354809e-07, 'epoch': 9.89}


100%|██████████| 21730/21730 [3:12:03<00:00,  1.89it/s]

{'train_runtime': 11523.8369, 'train_samples_per_second': 98.045, 'train_steps_per_second': 1.886, 'train_loss': 3.922839495908594, 'epoch': 10.0}


In [15]:
torch.cuda.empty_cache()

In [16]:
# Encode the conditioning tokens.
input_ids = tokenizer.encode("Jhana meditation is ", return_tensors="pt").cuda()
print(input_ids)

# Generate more tokens.
generated_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    temperature=0.5
)
generated_sequence = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
print(generated_sequence)

tensor([[483, 349,  80]], device='cuda:0')
Jhana meditation is what we call jhana. Jhana. because of the jhana factors are so strong and we can see that the jhana is very still and the first jhana is not the first jhana the third jhana is very strong and the second jhana is the first jhana and the fourth jhana and the second jhana is neither pleasant and the one pointedness that is neither pleasant nor not the piti and then the first jhana so there is no longer the first jhana is an object what it is one is at the third jhana which


In [30]:
# Encode the conditioning tokens.
input_ids = tokenizer.encode("To attain access concentration, we ", return_tensors="pt").cuda()
print(input_ids)

# Generate more tokens.
generated_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    temperature=0.5
)
generated_sequence = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
print(generated_sequence)

tensor([[1442,  651,  766,  322,    4,  104]], device='cuda:0')
To attain access concentration, we can experience the breath. as we breathe in, breathing out, and the breath is the breath, and the breath is the breath, the breath, the breath, the breath is the breath, the breath, the breath, the breath, the breath, the breath, the breath, the breath, the breath, the breath, the breath. the breath, the breath, the breath, the breath, the breath, the breath, the breath, the breath, the


In [31]:
# Assuming 'tokenizer' and 'model' are already defined

# Move model to GPU
model = model.cuda()

# Encode the conditioning tokens and move them to GPU
input_ids = tokenizer.encode("Jhana meditation is ", return_tensors="pt").cuda()
print(input_ids)

# Generate more tokens
generated_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    temperature=0.5
)

# Decode generated ids
generated_sequence = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


tensor([[483, 349,  80]], device='cuda:0')
Jhana meditation is a very helpful way to practice..... will be used to be a word., and then you've been talking about the word for jhana. or the jhana............................, and then you have to have to be able to have a sense of things and come from the different things. of the Jhana. and then you have to be


In [34]:
# Encode the conditioning tokens.
input_ids = tokenizer.encode("To practice jhana meditation, we sit ", return_tensors="pt").cuda()
print(input_ids)

# Generate more tokens.
generated_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    temperature=0.5
)
generated_sequence = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
print(generated_sequence)

tensor([[1442,  243,  257,  349,    4,  104,  332]], device='cuda:0')
To practice jhana meditation, we sit in a very deep meditation, in a way, in a way, in a way that we can see the whole world, as a way that we can see, we can see things., and see them as they are, and and they all are, and I can get to them, and they can be a little more, but the way we can see the way through the way that way we can go through the way it and the way you know it


In [35]:
# Assuming 'tokenizer' and 'model' are already defined

# Move model to GPU
model = model.cuda()

# Encode the conditioning tokens and move them to GPU
input_text = "In the first jhana, the experience is "
input_ids = tokenizer.encode(input_text, return_tensors="pt").cuda()
print(f"Input text: {input_text}")
print(f"Encoded input_ids: {input_ids} on device {input_ids.device}")

# Generate more tokens
generated_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    temperature=0.5
)
print(f"Raw generated ids: {generated_ids}")

# Decode generated ids
generated_sequence = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
print(f"Decoded generated sequence: {generated_sequence}")


Input text: In the first jhana, the experience is 
Encoded input_ids: tensor([[529,  71, 315, 257,   4,  71, 342,  80]], device='cuda:0') on device cuda:0
Raw generated ids: tensor([[ 529,   71,  315,  257,    4,   71,  342,   80,   71,  521,  552,   87,
           71,  315,  257,    4,   71,  521,  257,    4,   71,  686,  257,    4,
           85,   79,    3,   52,   71,  139,    5, 1942,   87,   71,  164,    6,
            6,  272,   80, 2997,  307,  121,  924,   85,  528,  909,   87,  322,
            6,  568,    6,  778,    6,    4,  272,   80, 2997,  307,  881,   85,
          528,    6,    6,    6,    6,   87, 1857,    6,  778,    6,    6,    6,
            6,  142,   71,  315,  257,    6,  698,   87,   71,  699,  483,    6,
            6,   87,   71,  521,  257,    6,  909,   87,  322,    6,    6,    6,
            6,    6,    6,   85]], device='cuda:0')
Decoded generated sequence: In the first jhana, the experience is the second factor of the first jhana, the second jhana, the 

In [20]:
input_text = "Jhana meditation is "
print(f"Input text: '{input_text}'")


Input text: 'Jhana meditation is '


In [21]:
tokens = tokenizer.tokenize(input_text)
print(f"Tokens: {tokens}")


Tokens: ['Jhana', 'meditation', 'is']


In [22]:
input_ids = tokenizer.encode(input_text, return_tensors="pt")
print(f"Encoded input_ids: {input_ids}")
decoded_tokens = [tokenizer.decode([id]) for id in input_ids[0]]
print(f"Decoded tokens from input_ids: {decoded_tokens}")


Encoded input_ids: tensor([[483, 349,  80]])
Decoded tokens from input_ids: ['Jhana', 'meditation', 'is']


In [23]:
print(f"Number of tokens: {len(tokens)}")
print(f"Number of input_ids: {input_ids.shape[1]}")


Number of tokens: 3
Number of input_ids: 3


In [24]:
print(f"Special tokens added: {tokenizer.build_inputs_with_special_tokens(tokens)}")


Special tokens added: ['Jhana', 'meditation', 'is']


In [25]:
attention_mask = tokenizer(input_text, return_tensors="pt")["attention_mask"]
print(f"Attention mask: {attention_mask}")


Attention mask: tensor([[1, 1, 1]])


In [26]:
print(f"Model device: {next(model.parameters()).device}")
print(f"Input ids device: {input_ids.device}")


Model device: cuda:0
Input ids device: cpu


In [27]:
# Ensure input_ids are on the same device as the model
input_ids = input_ids.to(next(model.parameters()).device)

# Now you can proceed with generating more tokens
generated_ids = model.generate(
    input_ids,
    max_length=100,
    do_sample=True,
    temperature=0.5
)

# Decode generated ids
generated_sequence = tokenizer.decode(generated_ids[0], clean_up_tokenization_spaces=True)
print(generated_sequence)


Jhana meditation is used to be translated in English as the sutras. and the Buddha says, and it's a very interesting thing., it's very important to say, it's a very, it's just a kind of a very, very subtle, very still, very, very, very, very subtle, very, very, very, very, very, very kind of a very, very subtle, very, very, very, very, very, very useful thing is


In [28]:
print(f"Model device: {next(model.parameters()).device}")
print(f"Input ids device: {input_ids.device}")


Model device: cuda:0
Input ids device: cuda:0


In [15]:
# Load model locally

#model = GPT2LMHeadModel.from_pretrained(pretrained_model_name_or_path=path)
#tokenizer = PreTrainedTokenizerFast(pretrained_model_name_or_path=path)